# Interpolasi

In [ ]:
import pandas as pd

# Membaca dataset
df_varietas = pd.read_excel('./dataset/split_data/varietas_tanah.xlsx')
df_lahan = pd.read_excel('./dataset/split_data/lahan_tanah.xlsx')
df_jenis_tanah = pd.read_excel('./dataset/bobot_ahp/bobot_jenis_tanah.xlsx')

# Fungsi interpolasi untuk variabel numerik
def interpolasi(x, min_val, max_val):
    if min_val <= x <= max_val:
        return 5
    elif 0 <= x < min_val:
        return 1 + ((x - 0) / (min_val - 0)) * (5 - 1)
    elif max_val < x <= (min_val+max_val):
        return 5 + ((x - max_val) / ((min_val + max_val) - max_val)) * (1 - 5)
    else:
        return 1

# Interpolasi untuk tekstur
def interpolasi_tekstur(x, value):
    if x in ['lempung', 'liat', 'berpasir', 'lempung berpasir']:
        return 5
    else:
        return 1

def interpolasi_jenis_tanah(x):
    if -1 <= x <= 0:
        return 5 + ((x - 0) / (-1 - 0)) * (4 - 5)
    elif 0 <= x <= 1:
        return 5 + ((x - 0) / (1 - 0)) * (4.5 - 5)


# Mengubah jenis tanah menjadi bobot berdasarkan data bobot jenis tanah
def ubah_jenis_tanah_ke_bobot(df_lahan, df_bobot_tanah):
    # Membuat dictionary untuk konversi
    bobot_dict = pd.Series(df_bobot_tanah.Weight.values, index=df_bobot_tanah.Label).to_dict()
    # Ubah kolom 'jenis_tanah' pada df_lahan menjadi nilai bobot
    df_lahan['jenis_tanah_bobot'] = df_lahan['jenis_tanah'].map(bobot_dict)
    return df_lahan

# Panggil fungsi ubah_jenis_tanah_ke_bobot
df_lahan_bobot = ubah_jenis_tanah_ke_bobot(df_lahan, df_jenis_tanah)

# Ubah varietas tanah ke bobot
def ubah_varietas_tanah_ke_bobot(df_varietas_tanah, df_bobot_tanah):
    # Membuat dictionary untuk konversi
    bobot_dict = pd.Series(df_bobot_tanah.Weight.values, index=df_bobot_tanah.Label).to_dict()
    # Ubah kolom 'jenis_tanah' pada df_varietas_tanah menjadi nilai bobot
    df_varietas_tanah['jenis_tanah_bobot'] = df_varietas_tanah['jenis_tanah'].map(bobot_dict)
    return df_varietas_tanah

# Panggil fungsi ubah_varietas_tanah_ke_bobot
df_varietas_bobot = ubah_varietas_tanah_ke_bobot(df_varietas, df_jenis_tanah)

# Hitung selisih bobot antara lahan dan varietas
def hitung_selisih_bobot(df_lahan_bobot, df_varietas_bobot, varietas):
    # Ambil bobot varietas
    varietas_bobot = df_varietas_bobot[df_varietas_bobot['jenis_varietas'] == varietas].iloc[0]['jenis_tanah_bobot']
    # Hitung selisih bobot
    df_lahan_bobot['selisih_bobot'] = df_lahan_bobot['jenis_tanah_bobot'] - varietas_bobot

    return df_lahan_bobot['selisih_bobot']


# Hitung skor tanah dengan menggunakan data selisih bobot
def hitung_skor_tanah(row, jenis_varietas):
    # Mengambil data varietas berdasarkan jenis_varietas
    var_data = df_varietas[df_varietas['jenis_varietas'] == jenis_varietas].iloc[0]
    
    # Mengambil nilai selisih_bobot untuk jenis_tanah
    hasil = {
        'ph': interpolasi(row['ph'], var_data['ph_min'], var_data['ph_max']),
        'kelembapan_tanah': interpolasi(row['kelembapan_tanah'], var_data['kelembapan_tanah_min'], var_data['kelembapan_tanah_max']),
        'jenis_tanah': interpolasi_jenis_tanah(row['selisih_bobot']),  # Menggunakan nilai dari selisih_bobot
        'tekstur_tanah': interpolasi_tekstur(row['tekstur_tanah'], var_data['tekstur_tanah'])
    }
    return hasil

# Panggil fungsi untuk varietas 
def hitung_skor_V1(row):
    return hitung_skor_tanah(row, 'V1')

def hitung_skor_V2(row):
    return hitung_skor_tanah(row, 'V2')

def hitung_skor_V3(row):
    return hitung_skor_tanah(row, 'V3')

def hitung_skor_V4(row):
    return hitung_skor_tanah(row, 'V4')

def hitung_skor_V5(row):
    return hitung_skor_tanah(row, 'V5')




# Bobot Utama

In [2]:
bobot_utama = pd.read_excel('./dataset/bobot_ahp/parameter_utama.xlsx')
bobot_utama_dict = bobot_utama.set_index('Label')['Weight'].to_dict()
bobot_utama_dict

{'iklim': 0.4053875480953168,
 'topografi': 0.1139723667233914,
 'tanah': 0.4806400851812919}

# Bobot Parameter Tanah

In [3]:
bobot_tanah = pd.read_excel('./dataset/bobot_ahp/parameter_tanah.xlsx')
bobot_tanah_dict = bobot_tanah.set_index('Label')['Weight'].to_dict()
bobot_tanah_dict

{'ph': 0.3814374052035476,
 'kelembapan_tanah': 0.09848671453159402,
 'jenis_tanah': 0.4333965668873818,
 'tekstur_tanah': 0.08667931337747636}

# Varietas 1

In [ ]:

df_lahan_bobot['selisih_bobot'] = hitung_selisih_bobot(df_lahan_bobot, df_varietas_bobot, 'V1')

lahan_scores = df_lahan_bobot.apply(hitung_skor_V1, axis=1)

lahan_tanah_V1 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)


lahan_tanah_V1

,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah
0,Bakung,5.0,4.626667,4.765187,1.0
1,Binangun,5.0,4.626667,4.658762,1.0
2,Doko,5.0,3.640000,5.000000,5.0
3,Gandusari,5.0,3.640000,4.880826,5.0
4,Garum,5.0,3.640000,4.716732,5.0
5,Kademangan,5.0,4.160000,4.765187,5.0
6,Kanigoro,5.0,4.626667,4.716732,5.0
7,Kesamben,5.0,4.986667,4.658762,5.0
8,Nglegok,5.0,3.640000,4.880826,5.0
9,Panggungrejo,5.0,4.626667,4.765187,1.0


### SUM (Perkalian skor V1 dengan bobot AHP)

In [6]:
# Variabel untuk menyimpan hasil perkalian dengan bobot
skor_tanah_V1 = lahan_tanah_V1.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']:
    skor_tanah_V1[kolom] = skor_tanah_V1[kolom] * bobot_tanah_dict[kolom]

# Menambahkan kolom 'total_skor' yang berisi jumlah dari kolom yang telah dikalikan bobot
skor_tanah_V1['total_skor'] = skor_tanah_V1[['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']].sum(axis=1)

skor_tanah_V1


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor
0,Bakung,1.907187,0.455665,2.065215,0.086679,4.514747
1,Binangun,1.907187,0.455665,2.019091,0.086679,4.468623
2,Doko,1.907187,0.358492,2.166983,0.433397,4.866058
3,Gandusari,1.907187,0.358492,2.115333,0.433397,4.814409
4,Garum,1.907187,0.358492,2.044216,0.433397,4.743291
5,Kademangan,1.907187,0.409705,2.065215,0.433397,4.815504
6,Kanigoro,1.907187,0.455665,2.044216,0.433397,4.840464
7,Kesamben,1.907187,0.491120,2.019091,0.433397,4.850795
8,Nglegok,1.907187,0.358492,2.115333,0.433397,4.814409
9,Panggungrejo,1.907187,0.455665,2.065215,0.086679,4.514747


### Perkalian skor total Tanah V1 dengan bobot AHP tanah

In [7]:

total_skor_tanah_V1 = skor_tanah_V1.copy()
total_skor_tanah_V1['total_skor_tanah_V1'] = total_skor_tanah_V1['total_skor'] * bobot_utama_dict['tanah']
total_skor_tanah_V1


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor,total_skor_tanah_V1
0,Bakung,1.907187,0.455665,2.065215,0.086679,4.514747,2.169968
1,Binangun,1.907187,0.455665,2.019091,0.086679,4.468623,2.147799
2,Doko,1.907187,0.358492,2.166983,0.433397,4.866058,2.338823
3,Gandusari,1.907187,0.358492,2.115333,0.433397,4.814409,2.313998
4,Garum,1.907187,0.358492,2.044216,0.433397,4.743291,2.279816
5,Kademangan,1.907187,0.409705,2.065215,0.433397,4.815504,2.314524
6,Kanigoro,1.907187,0.455665,2.044216,0.433397,4.840464,2.326521
7,Kesamben,1.907187,0.491120,2.019091,0.433397,4.850795,2.331487
8,Nglegok,1.907187,0.358492,2.115333,0.433397,4.814409,2.313998
9,Panggungrejo,1.907187,0.455665,2.065215,0.086679,4.514747,2.169968


# Varietas 2

In [8]:
# Menambahkan selisih bobot ke df_lahan
df_lahan_bobot['selisih_bobot'] = hitung_selisih_bobot(df_lahan_bobot, df_varietas_bobot, 'V2')

lahan_scores = df_lahan_bobot.apply(hitung_skor_V2, axis=1)

lahan_tanah_V2 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_tanah_V2

,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah
0,Bakung,5.0,5.00,4.884360,1.0
1,Binangun,5.0,5.00,4.777935,1.0
2,Doko,5.0,4.96,4.940413,5.0
3,Gandusari,5.0,4.96,5.000000,5.0
4,Garum,5.0,4.96,4.835906,5.0
5,Kademangan,5.0,5.00,4.884360,5.0
6,Kanigoro,5.0,5.00,4.835906,5.0
7,Kesamben,5.0,5.00,4.777935,5.0
8,Nglegok,5.0,4.96,5.000000,5.0
9,Panggungrejo,5.0,5.00,4.884360,1.0


### SUM (Perkalian skor V2 dengan bobot AHP)

In [9]:
# Variabel untuk menyimpan hasil perkalian dengan bobot
skor_tanah_V2 = lahan_tanah_V2.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']:
    skor_tanah_V2[kolom] = skor_tanah_V2[kolom] * bobot_tanah_dict[kolom]

# Menambahkan kolom 'total_skor' yang berisi jumlah dari kolom yang telah dikalikan bobot
skor_tanah_V2['total_skor'] = skor_tanah_V2[['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']].sum(axis=1)

skor_tanah_V2


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor
0,Bakung,1.907187,0.492434,2.116865,0.086679,4.603165
1,Binangun,1.907187,0.492434,2.070741,0.086679,4.557041
2,Doko,1.907187,0.488494,2.141158,0.433397,4.970236
3,Gandusari,1.907187,0.488494,2.166983,0.433397,4.996061
4,Garum,1.907187,0.488494,2.095865,0.433397,4.924943
5,Kademangan,1.907187,0.492434,2.116865,0.433397,4.949882
6,Kanigoro,1.907187,0.492434,2.095865,0.433397,4.928882
7,Kesamben,1.907187,0.492434,2.070741,0.433397,4.903758
8,Nglegok,1.907187,0.488494,2.166983,0.433397,4.996061
9,Panggungrejo,1.907187,0.492434,2.116865,0.086679,4.603165


### Perkalian skor total Tanah V2 dengan bobot AHP tanah

In [10]:

total_skor_tanah_V2 = skor_tanah_V2.copy()
total_skor_tanah_V2['total_skor_tanah_V2'] = total_skor_tanah_V2['total_skor'] * bobot_utama_dict['tanah']
total_skor_tanah_V2


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor,total_skor_tanah_V2
0,Bakung,1.907187,0.492434,2.116865,0.086679,4.603165,2.212466
1,Binangun,1.907187,0.492434,2.070741,0.086679,4.557041,2.190296
2,Doko,1.907187,0.488494,2.141158,0.433397,4.970236,2.388895
3,Gandusari,1.907187,0.488494,2.166983,0.433397,4.996061,2.401307
4,Garum,1.907187,0.488494,2.095865,0.433397,4.924943,2.367125
5,Kademangan,1.907187,0.492434,2.116865,0.433397,4.949882,2.379112
6,Kanigoro,1.907187,0.492434,2.095865,0.433397,4.928882,2.369018
7,Kesamben,1.907187,0.492434,2.070741,0.433397,4.903758,2.356943
8,Nglegok,1.907187,0.488494,2.166983,0.433397,4.996061,2.401307
9,Panggungrejo,1.907187,0.492434,2.116865,0.086679,4.603165,2.212466


# Varietas V3

In [11]:
# Menambahkan selisih bobot ke df_lahan
df_lahan_bobot['selisih_bobot'] = hitung_selisih_bobot(df_lahan_bobot, df_varietas_bobot, 'V3')

lahan_scores = df_lahan_bobot.apply(hitung_skor_V3, axis=1)

lahan_tanah_V3 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_tanah_V3

,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah
0,Bakung,5.0,4.626667,4.884360,1.0
1,Binangun,5.0,4.626667,4.777935,1.0
2,Doko,5.0,3.640000,4.940413,5.0
3,Gandusari,5.0,3.640000,5.000000,5.0
4,Garum,5.0,3.640000,4.835906,5.0
5,Kademangan,5.0,4.160000,4.884360,5.0
6,Kanigoro,5.0,4.626667,4.835906,5.0
7,Kesamben,5.0,4.986667,4.777935,5.0
8,Nglegok,5.0,3.640000,5.000000,5.0
9,Panggungrejo,5.0,4.626667,4.884360,1.0


### SUM (Perkalian skor V3 dengan bobot AHP)

In [12]:
# Variabel untuk menyimpan hasil perkalian dengan bobot
skor_tanah_V3 = lahan_tanah_V3.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']:
    skor_tanah_V3[kolom] = skor_tanah_V3[kolom] * bobot_tanah_dict[kolom]

# Menambahkan kolom 'total_skor' yang berisi jumlah dari kolom yang telah dikalikan bobot
skor_tanah_V3['total_skor'] = skor_tanah_V3[['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']].sum(axis=1)

skor_tanah_V3


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor
0,Bakung,1.907187,0.455665,2.116865,0.086679,4.566396
1,Binangun,1.907187,0.455665,2.070741,0.086679,4.520272
2,Doko,1.907187,0.358492,2.141158,0.433397,4.840233
3,Gandusari,1.907187,0.358492,2.166983,0.433397,4.866058
4,Garum,1.907187,0.358492,2.095865,0.433397,4.794940
5,Kademangan,1.907187,0.409705,2.116865,0.433397,4.867153
6,Kanigoro,1.907187,0.455665,2.095865,0.433397,4.892114
7,Kesamben,1.907187,0.491120,2.070741,0.433397,4.902445
8,Nglegok,1.907187,0.358492,2.166983,0.433397,4.866058
9,Panggungrejo,1.907187,0.455665,2.116865,0.086679,4.566396


### Perkalian skor total Tanah V3 dengan bobot AHP tanah

In [13]:

total_skor_tanah_V3 = skor_tanah_V3.copy()
total_skor_tanah_V3['total_skor_tanah_V3'] = total_skor_tanah_V3['total_skor'] * bobot_utama_dict['tanah']
total_skor_tanah_V3


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor,total_skor_tanah_V3
0,Bakung,1.907187,0.455665,2.116865,0.086679,4.566396,2.194793
1,Binangun,1.907187,0.455665,2.070741,0.086679,4.520272,2.172624
2,Doko,1.907187,0.358492,2.141158,0.433397,4.840233,2.326410
3,Gandusari,1.907187,0.358492,2.166983,0.433397,4.866058,2.338823
4,Garum,1.907187,0.358492,2.095865,0.433397,4.794940,2.304641
5,Kademangan,1.907187,0.409705,2.116865,0.433397,4.867153,2.339349
6,Kanigoro,1.907187,0.455665,2.095865,0.433397,4.892114,2.351346
7,Kesamben,1.907187,0.491120,2.070741,0.433397,4.902445,2.356311
8,Nglegok,1.907187,0.358492,2.166983,0.433397,4.866058,2.338823
9,Panggungrejo,1.907187,0.455665,2.116865,0.086679,4.566396,2.194793


# Varietas V4

In [14]:
# Menambahkan selisih bobot ke df_lahan
df_lahan_bobot['selisih_bobot'] = hitung_selisih_bobot(df_lahan_bobot, df_varietas_bobot, 'V4')

lahan_scores = df_lahan_bobot.apply(hitung_skor_V4, axis=1)

lahan_tanah_V4 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_tanah_V4


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah
0,Bakung,5.0,4.626667,4.884360,1.0
1,Binangun,5.0,4.626667,4.777935,1.0
2,Doko,5.0,3.640000,4.940413,5.0
3,Gandusari,5.0,3.640000,5.000000,5.0
4,Garum,5.0,3.640000,4.835906,5.0
5,Kademangan,5.0,4.160000,4.884360,5.0
6,Kanigoro,5.0,4.626667,4.835906,5.0
7,Kesamben,5.0,4.986667,4.777935,5.0
8,Nglegok,5.0,3.640000,5.000000,5.0
9,Panggungrejo,5.0,4.626667,4.884360,1.0


### SUM (Perkalian skor V4 dengan bobot AHP)

In [15]:
# Variabel untuk menyimpan hasil perkalian dengan bobot
skor_tanah_V4 = lahan_tanah_V4.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']:
    skor_tanah_V4[kolom] = skor_tanah_V4[kolom] * bobot_tanah_dict[kolom]

# Menambahkan kolom 'total_skor' yang berisi jumlah dari kolom yang telah dikalikan bobot
skor_tanah_V4['total_skor'] = skor_tanah_V4[['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']].sum(axis=1)

skor_tanah_V4


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor
0,Bakung,1.907187,0.455665,2.116865,0.086679,4.566396
1,Binangun,1.907187,0.455665,2.070741,0.086679,4.520272
2,Doko,1.907187,0.358492,2.141158,0.433397,4.840233
3,Gandusari,1.907187,0.358492,2.166983,0.433397,4.866058
4,Garum,1.907187,0.358492,2.095865,0.433397,4.794940
5,Kademangan,1.907187,0.409705,2.116865,0.433397,4.867153
6,Kanigoro,1.907187,0.455665,2.095865,0.433397,4.892114
7,Kesamben,1.907187,0.491120,2.070741,0.433397,4.902445
8,Nglegok,1.907187,0.358492,2.166983,0.433397,4.866058
9,Panggungrejo,1.907187,0.455665,2.116865,0.086679,4.566396


### Perkalian skor total Tanah V4 dengan bobot AHP tanah

In [16]:

total_skor_tanah_V4 = skor_tanah_V4.copy()
total_skor_tanah_V4['total_skor_tanah_V4'] = total_skor_tanah_V4['total_skor'] * bobot_utama_dict['tanah']
total_skor_tanah_V4


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor,total_skor_tanah_V4
0,Bakung,1.907187,0.455665,2.116865,0.086679,4.566396,2.194793
1,Binangun,1.907187,0.455665,2.070741,0.086679,4.520272,2.172624
2,Doko,1.907187,0.358492,2.141158,0.433397,4.840233,2.326410
3,Gandusari,1.907187,0.358492,2.166983,0.433397,4.866058,2.338823
4,Garum,1.907187,0.358492,2.095865,0.433397,4.794940,2.304641
5,Kademangan,1.907187,0.409705,2.116865,0.433397,4.867153,2.339349
6,Kanigoro,1.907187,0.455665,2.095865,0.433397,4.892114,2.351346
7,Kesamben,1.907187,0.491120,2.070741,0.433397,4.902445,2.356311
8,Nglegok,1.907187,0.358492,2.166983,0.433397,4.866058,2.338823
9,Panggungrejo,1.907187,0.455665,2.116865,0.086679,4.566396,2.194793


# Varietas V5

In [17]:
# Menambahkan selisih bobot ke df_lahan
df_lahan_bobot['selisih_bobot'] = hitung_selisih_bobot(df_lahan_bobot, df_varietas_bobot, 'V5')

lahan_scores = df_lahan_bobot.apply(hitung_skor_V5, axis=1)

lahan_tanah_V5 = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

lahan_tanah_V5

,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah
0,Bakung,5.0,4.626667,4.765187,1.0
1,Binangun,5.0,4.626667,4.658762,1.0
2,Doko,5.0,3.640000,5.000000,5.0
3,Gandusari,5.0,3.640000,4.880826,5.0
4,Garum,5.0,3.640000,4.716732,5.0
5,Kademangan,5.0,4.160000,4.765187,5.0
6,Kanigoro,5.0,4.626667,4.716732,5.0
7,Kesamben,5.0,4.986667,4.658762,5.0
8,Nglegok,5.0,3.640000,4.880826,5.0
9,Panggungrejo,5.0,4.626667,4.765187,1.0


### SUM (Perkalian skor V5 dengan bobot AHP)

In [18]:
# Variabel untuk menyimpan hasil perkalian dengan bobot
skor_tanah_V5 = lahan_tanah_V5.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']:
    skor_tanah_V5[kolom] = skor_tanah_V5[kolom] * bobot_tanah_dict[kolom]

# Menambahkan kolom 'total_skor' yang berisi jumlah dari kolom yang telah dikalikan bobot
skor_tanah_V5['total_skor'] = skor_tanah_V5[['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']].sum(axis=1)

skor_tanah_V5


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor
0,Bakung,1.907187,0.455665,2.065215,0.086679,4.514747
1,Binangun,1.907187,0.455665,2.019091,0.086679,4.468623
2,Doko,1.907187,0.358492,2.166983,0.433397,4.866058
3,Gandusari,1.907187,0.358492,2.115333,0.433397,4.814409
4,Garum,1.907187,0.358492,2.044216,0.433397,4.743291
5,Kademangan,1.907187,0.409705,2.065215,0.433397,4.815504
6,Kanigoro,1.907187,0.455665,2.044216,0.433397,4.840464
7,Kesamben,1.907187,0.491120,2.019091,0.433397,4.850795
8,Nglegok,1.907187,0.358492,2.115333,0.433397,4.814409
9,Panggungrejo,1.907187,0.455665,2.065215,0.086679,4.514747


### Perkalian skor total Tanah V5 dengan bobot AHP tanah

In [19]:

total_skor_tanah_V5 = skor_tanah_V5.copy()
total_skor_tanah_V5['total_skor_tanah_V5'] = total_skor_tanah_V5['total_skor'] * bobot_utama_dict['tanah']
total_skor_tanah_V5


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor,total_skor_tanah_V5
0,Bakung,1.907187,0.455665,2.065215,0.086679,4.514747,2.169968
1,Binangun,1.907187,0.455665,2.019091,0.086679,4.468623,2.147799
2,Doko,1.907187,0.358492,2.166983,0.433397,4.866058,2.338823
3,Gandusari,1.907187,0.358492,2.115333,0.433397,4.814409,2.313998
4,Garum,1.907187,0.358492,2.044216,0.433397,4.743291,2.279816
5,Kademangan,1.907187,0.409705,2.065215,0.433397,4.815504,2.314524
6,Kanigoro,1.907187,0.455665,2.044216,0.433397,4.840464,2.326521
7,Kesamben,1.907187,0.491120,2.019091,0.433397,4.850795,2.331487
8,Nglegok,1.907187,0.358492,2.115333,0.433397,4.814409,2.313998
9,Panggungrejo,1.907187,0.455665,2.065215,0.086679,4.514747,2.169968
